In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers, mixed_precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os 

import optuna
import wandb
import gc

In [ ]:
mixed_precision.set_global_policy("mixed_float16")

gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("TensorFlow is using the GPU \n", gpus)
else:
    print("No GPU detected.")
    
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
from wandb.integration.keras import WandbMetricsLogger

wandb.require("core")
wandb.login()

In [ ]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (128, 128))
    img = img / 255.0
    return img, label

def make_dataset(paths, labels, batch_size = 32, shuffle = False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
# Se cargan los datos
# Se debe de realizar una transformación en los datos, convirtiéndolos de -1/1 a 0/1:


ds = pd.read_csv("/tf/Face-Recognition/CelebA/list_attr_celeba.txt", sep = r"\s+", skiprows = 1)
ds.iloc[:, 0:39] = (ds.iloc[:, 0:39] == 1).astype("int32")
#ds

img_dir = "/img_align_celeba"

ds["image_path"] = ds.index.map(lambda x: os.path.join(img_dir, x))
ds.reset_index(inplace = True)
ds.rename(columns = {"index" : "image"}, inplace = True)
ds.head()

df_split = pd.read_csv(
    "/tf/Face-Recognition/CelebA/Eval/list_eval_partition.txt", 
    sep = r"\s+",
    names = ["image", "partition"])

ds = ds.merge(df_split, on = "image")
ds.head()

df_train = ds[ds["partition"] == 0]
df_val   = ds[ds["partition"] == 1]
df_test  = ds[ds["partition"] == 2]

train_ds = make_dataset(
    df_train["image_path"].values, 
    df_train.iloc[:, 1:41].values,
    shuffle = True
)

val_ds = make_dataset(
    df_val["image_path"].values,
    df_val.iloc[:, 1:41].values,
    shuffle = False
)

test_ds = make_dataset(
    df_test["image_path"].values,
    df_test.iloc[:, 1:41].values,
    shuffle = False
)

In [ ]:
df_train_small = df_train.sample(frac = 0.05, random_state = 5)
df_val_small = df_val.sample(frac = 0.05, random_state = 5)
df_test_small = df_test.sample(frac = 0.05, random_state = 5)

train_ds_small = make_dataset(
    df_train_small["image_path"].values,
    df_train_small.iloc[:, 1:41].values,
    shuffle = True,
)

val_ds_small = make_dataset(
    df_val_small["image_path"].values,
    df_val_small.iloc[:, 1:41].values,
)

test_ds_small = make_dataset(
    df_test_small["image_path"].values,
    df_test_small.iloc[:, 1:41].values,
)

In [ ]:
MobileNetV2 = keras.applications.MobileNetV2(
    weights = "imagenet",
    input_shape = (128, 128, 3),
    include_top = False)

In [ ]:
MobileNetV2.trainable = False

In [ ]:
inputs = keras.Input(shape = (128, 128, 3))
x = MobileNetV2(inputs, training = False)
x = layers.Flatten()(x)
x = layers.Dense(256, activation = "leaky_relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128,activation = "leaky_relu")(x)
x = layers.Dropout(0.15)(x)
x = layers.Dense(64, activation = "relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(40, activation = "softmax", dtype = "float32")(x)
model = keras.Model(inputs, outputs)

model.compile(loss = "categorical_crossentropy", optimizer = "rmsprop", metrics = ["accuracy"])

model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 10, restore_best_weights = True)
lr_reduction = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 7)

In [ ]:
lr = 1e-4
optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr) 

In [ ]:
wandb.init(
        project = "Conv2D-ResNetV2-Based-Trials-Exp-Series1.0",
        name = "Trial_1",
        reinit = True,
        config = {
            "activation": "leaky_relu, relu",
            "n_layers": 3,
            "learning_rate": lr,
            "optimizer": "RMSProp"
        }
    )

In [ ]:
history = model.fit(
    train_ds_small, 
    validation_data = val_ds_small,
    epochs = 200,
    verbose = 1, 
    callbacks = [WandbMetricsLogger(log_freq = 5), early_stopping, lr_reduction]
        )

In [ ]:
tf.keras.backend.clear_session()
wandb.finish()
gc.collect()